In [0]:
def read_file(file_path):
    with open(file_path) as f:
        str_text = f.read()
    
    return str_text

In [0]:
import spacy

In [0]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [0]:
nlp.max_length = 1198623

In [0]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [6]:
from google.colab import files
file = files.upload()
d = read_file('moby_dick_four_chapters.txt')

Saving moby_dick_four_chapters.txt to moby_dick_four_chapters (1).txt


In [0]:
#d

In [0]:
tokens = separate_punc(d)

In [9]:
len(tokens)

11338

In [0]:
#tokens

In [0]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
  seq = tokens[i-train_len:i]
  
  text_sequences.append(seq)

In [12]:
type(text_sequences)

list

In [0]:
#text_sequences

In [14]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [15]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer()

In [0]:
tokenizer.fit_on_texts(text_sequences)

In [0]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [20]:
sequences[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 315,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2712,
 14,
 24]

In [0]:
#tokenizer.index_word

In [0]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [0]:
#tokenizer.word_counts

In [0]:
vocabulary_size = len(tokenizer.word_counts)

In [0]:
import numpy as np

In [0]:
sequences = np.array(sequences)

In [26]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [0]:
from keras.utils import to_categorical

In [0]:
X = sequences[:, :-1]

In [0]:
y = sequences[:, -1]

In [0]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [0]:
seq_len=X.shape[1]

In [0]:
import keras

In [0]:
from keras.models import Sequential

In [0]:
from keras.layers import Dense, LSTM, Embedding

In [0]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [36]:
model = create_model(vocabulary_size+1,seq_len)

W0618 23:20:16.214856 140125796800384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0618 23:20:16.237597 140125796800384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0618 23:20:16.243434 140125796800384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0618 23:20:16.799396 140125796800384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0618 23:20:16.824729 140125796800384 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 150)           105600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_2 (Dense)              (None, 2718)              410418    
Total params: 787,218
Trainable params: 787,218
Non-trainable params: 0
_________________________________________________________________


In [0]:
from pickle import dump, load

In [38]:
model.fit(X, y, batch_size=128, epochs=2, verbose=1)

W0618 23:20:21.118521 140125796800384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0618 23:20:22.584414 140125796800384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/2
11312/11312 [==============================] - 22s 2ms/step - loss: 6.8652 - acc: 0.0504
Epoch 2/2
11312/11312 [==============================] - 20s 2ms/step - loss: 6.3846 - acc: 0.0529


In [0]:
model.save('my_mobidick_model.h5')

In [0]:
dump(tokenizer, open('my_simpletokenizer', 'wb'))

In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [0]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [0]:
#text_sequences[0]

In [0]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [0]:
random_seed_text = text_sequences[random_pick]

In [46]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [0]:
seed_text = ' '.join(random_seed_text)

In [48]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [49]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the'

In [0]:
from keras.models import load_model

In [0]:
file = files.upload()
model = load_model('epochBIG.h5')

Saving epochBIG.h5 to epochBIG (2).h5


In [0]:
file = files.upload()
tokenizer = load(open('epochBIG', 'rb'))

Saving epochBIG to epochBIG


In [0]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

"to be seen there was no bad olfactories my own letter was cheerily listening over his hearers who 's more can go have a wearing answer to accumulate a vow and do him not they to think of his lances stubb sperm by the vast man 's sign by the"